In [1]:
from cassandra.cluster import Cluster

In [2]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [7]:
result = session.execute("""CREATE KEYSPACE shop
    WITH replication  = {
        'class': 'SimpleStrategy',
        'replication_factor': 1
        } ;""").one()
if result:
    print(result[0])

AlreadyExists: Keyspace 'shop' already exists

In [8]:
result = session.execute("""USE shop""").one()
if result:
    print(result[0])

In [19]:
result = session.execute("""
CREATE TABLE IF NOT EXISTS item (
  category text,
  price int,
  model text,
  producer text,
  properties map<text, text>,
  PRIMARY KEY (category, price)
)
WITH CLUSTERING ORDER BY (price ASC);""").one()
if result:
    print(result[0])

In [22]:
session.execute("""
INSERT INTO item (
    category, price, model, producer, properties
) VALUES (
          'Phone',
          900,
          'iPhone 14',
          'Apple',
          {
              'display': '6.1'
              }
         );
""")
session.execute("""
INSERT INTO item (
    category, price, model, producer, properties
) VALUES (
          'Phone',
          800,
          'iPhone 13',
          'Apple',
          {
              'display': '6.1'
              }
         );
""")
session.execute("""
INSERT INTO item (
    category, price, model, producer, properties
) VALUES (
          'Phone',
          850,
          'Galaxy s23',
          'Samsung',
          {
              'display': '6.2'
              }
         );
""")
session.execute("""
INSERT INTO item (
    category, price, model, producer, properties
) VALUES (
          'Watch',
          400,
          'Series 8',
          'Apple',
          {
              'feature': 'ECG',
              'material': 'aluminium'
              }
         );
""")
session.execute("""
INSERT INTO item (
    category, price, model, producer, properties
) VALUES (
          'Watch',
          350,
          'Galaxy Watch5',
          'Samsung',
          {
              'feature': 'Sleep Coaching',
              'material': 'aluminium'
              }
         );
""")
session.execute("""
INSERT INTO item (
    category, price, model, producer, properties
) VALUES (
          'Headphones',
          180,
          'AirPods 3',
          'Apple',
          {
              'feature': 'Force sensor',
              'connectivity': 'Bluetooth 5.0'
              }
         );
""")
session.execute("""
INSERT INTO item (
    category, price, model, producer, properties
) VALUES (
          'Headphones',
          125,
          'Galaxy Buds2 Pro',
          'Samsung',
          {
              'feature': 'IPX7 water-resistant',
              'connectivity': 'Bluetooth 5.0'
              }
         );
""")

1) Напишіть запит, який показує структуру створеної таблиці (команда DESCRIBE)


In [30]:
result = session.execute("""DESCRIBE TABLE item;""").one()
if result:
    print(*result)

shop table item CREATE TABLE shop.item (
    category text,
    price int,
    model text,
    producer text,
    properties map<text, text>,
    PRIMARY KEY (category, price)
) WITH CLUSTERING ORDER BY (price ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


2) Напишіть запит, який виводить усі товари в певній категорії відсортовані за ціною


In [35]:
result = session.execute("""
SELECT *
FROM item
WHERE category = 'Watch'
ORDER BY price;""")
if result:
    for row in result:
        print(*row, '\n')

Watch 350 Galaxy Watch5 Samsung {'feature': 'Sleep Coaching', 'material': 'aluminium'} 

Watch 400 Series 8 Apple {'feature': 'ECG', 'material': 'aluminium'} 



3.1) Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії (тут де треба замість індексу використайте Matirialized view): назва

In [53]:
session.execute("""CREATE INDEX IF NOT EXISTS model_idx ON shop.item(model);""")

In [52]:
result = session.execute("""
SELECT *
FROM item
WHERE category = 'Watch' and model = 'Series 8';""")
if result:
    for row in result:
        print(*row, '\n')

Watch 400 Series 8 Apple {'feature': 'ECG', 'material': 'aluminium'} 



3.2) Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії (тут де треба замість індексу використайте Matirialized view): ціна (в проміжку)

In [38]:
result = session.execute("""
SELECT *
FROM item
WHERE   category = 'Watch'
    and price > 300 and price < 400;""")
if result:
    for row in result:
        print(*row, '\n')

Watch 350 Galaxy Watch5 Samsung {'feature': 'Sleep Coaching', 'material': 'aluminium'} 



3.3) Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії (тут де треба замість індексу використайте Matirialized view): ціна та виробник


In [39]:
session.execute("""
CREATE MATERIALIZED VIEW shop.item_by_price_producer
AS SELECT category, price, model, producer, properties
FROM shop.item
WHERE category is not null
  and price is not null
  and producer is not null
PRIMARY KEY ( category, price, producer );""")


In [41]:
result = session.execute("""
SELECT *
FROM item_by_price_producer
WHERE   category = 'Phone'
    and price = 800
    and producer = 'Apple';""")
if result:
    for row in result:
        print(*row, '\n')

Phone 800 Apple iPhone 13 {'display': '6.1'} 



4.1) Напишіть запити, які вибирають товари за:
наявність певних характеристик

In [42]:
session.execute("""
CREATE INDEX IF NOT EXISTS properties_idx
ON shop.item ( KEYS (properties) );
""")

In [43]:
result = session.execute("""
SELECT *
FROM item
WHERE properties contains key 'material';""")
if result:
    for row in result:
        print(*row, '\n')

Watch 350 Galaxy Watch5 Samsung {'feature': 'Sleep Coaching', 'material': 'aluminium'} 

Watch 400 Series 8 Apple {'feature': 'ECG', 'material': 'aluminium'} 



4.2) Напишіть запити, які вибирають товари за:
певна характеристика та її значення

In [44]:
session.execute("""
CREATE INDEX properties_idx_2
ON shop.item (ENTRIES(properties));
""")

In [45]:
result = session.execute("""
SELECT *
FROM item
WHERE properties['display'] = '6.1';""")
if result:
    for row in result:
        print(*row, '\n')

Phone 800 iPhone 13 Apple {'display': '6.1'} 

Phone 900 iPhone 14 Apple {'display': '6.1'} 



5.1) Оновити опис товару: змінить існуючі значення певної характеристики

In [46]:
session.execute("""
UPDATE item
SET properties['material'] = 'steel'
WHERE category = 'Watch' and price = 350;
""")

In [47]:
result = session.execute("""
SELECT *
FROM item
WHERE category = 'Watch' and price = 350;""")
if result:
    for row in result:
        print(*row, '\n')

Watch 350 Galaxy Watch5 Samsung {'feature': 'Sleep Coaching', 'material': 'steel'} 



5.2) Оновити опис товару: додайте нові властивості (характеристики) товару

In [48]:
session.execute("""
UPDATE item
SET properties = properties + {'colour': 'red'}
WHERE category = 'Watch' and price = 350;
""")

In [49]:
result = session.execute("""
SELECT *
FROM item
WHERE category = 'Watch' and price = 350;""")
if result:
    for row in result:
        print(*row, '\n')

Watch 350 Galaxy Watch5 Samsung {'colour': 'red', 'feature': 'Sleep Coaching', 'material': 'steel'} 



5.3) Оновити опис товару: видалить характеристику товару


In [50]:
session.execute("""
DELETE properties['feature'] FROM item
WHERE category = 'Watch' and price = 350;
""")

In [51]:
result = session.execute("""
SELECT *
FROM item
WHERE category = 'Watch' and price = 350;""")
if result:
    for row in result:
        print(*row, '\n')

Watch 350 Galaxy Watch5 Samsung {'colour': 'red', 'material': 'steel'} 

